In [2]:
from sklearn.model_selection import train_test_split

import numpy as np
from sklearn import svm
import pandas as pd 
from mass import Modify_Kernel as MKernel
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import KernelPCA
from sklearn.ensemble import RandomForestClassifier

import pandas as pd
from sklearn.decomposition import KernelPCA
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, cross_val_score, KFold

In [3]:
param_value = None
data_stats = None

In [4]:
abalone = pd.read_csv("data/abalone_test.csv",header = None)
abalone = {"X":np.array(abalone.iloc[:, :-1]),
            "Y":np.array(abalone.iloc[:, -1])}




In [5]:
def make_missing(data, rate = 0.1, type = "mcar"):
    missing_rate = rate  # 10% of the data will be missing
    # Calculate the number of elements to set as missing
    total_elements = data.size
    missing_elements = int(total_elements * missing_rate)

    # Create a random mask
    #np.random.seed(1)

    if type == "mcar":
        mask_indices = np.random.choice(total_elements, missing_elements, replace=False)

        # Convert flat indices to multi-dimensional indices
        multi_indices = np.unravel_index(mask_indices, data.shape)

        # Set selected elements to NaN
        data[multi_indices] = np.nan
    elif type == "mnar":
        for col in range(data.shape[1]):
            column_data = data[:, col]
            median_value = np.percentile(column_data, 60)
            #print(len(upper_quantile_indices))
            upper_quantile_indices = np.where(column_data > median_value)[0]
            missingnum = int(missing_elements/data.shape[1])

            selected_indices = np.random.choice(upper_quantile_indices,missingnum , replace=False)
            data[selected_indices, col] = np.nan


    return data

In [6]:
data_stats = None

## Syn 1 (Simple)

In [9]:
## M0 
run_test(syn_1,0.05)
run_test(syn_1,0.2)
run_test(syn_1,0.5)

NameError: name 'syn_1' is not defined

In [ ]:
## Modify
run_test(syn_1,0.05)
run_test(syn_1,0.2)
run_test(syn_1,0.5)

## MNAR

### Banknote

In [ ]:

run_test(banknote,0.05,"mnar")
run_test(syn_1,0.05,"mnar","mean")


# run_test(banknote,0.1,"mnar")
# run_test(banknote,0.1,"mnar","mean")


# run_test(banknote,0.3,"mnar")
# run_test(banknote,0.3,"mnar","mean")

### abalone - Regression

In [12]:
## Modify
run_test_reg(abalone,0.05,"mnar")
# run_test_reg(abalone,0.1,"mnar")
# run_test_reg(abalone,0.3,"mnar")

TypeError: string indices must be integers, not 'str'

In [8]:
from sklearn.impute import SimpleImputer
def run_test(data,missing_rate,mtype = "mcar",model = "mass",data_stats = None):
    X = data["X"]
    Y = data["Y"]
    train_X, test_X, train_Y, test_Y = train_test_split(X, Y, test_size=0.2, random_state=42)


    train_X = make_missing(train_X,rate = missing_rate,type = mtype)
    test_X = make_missing(test_X,rate = missing_rate,type = mtype)
    
    if model == "mass":
        kernal = MKernel(None, data_stats)
        kernal.set_nbins(param_value)


        train_mod, test_mod = kernal.build_model(train_X, test_X)  # this does the pre-processing step

        sim_train = kernal.transform(train_mod)
        sim_test = kernal.transform(test_mod,train_mod)  # row = train, col = test

        # Configure Kernel PCA for precomputed kernels
        kpca = KernelPCA(kernel='precomputed')

        # Transform data using Kernel PCA
        X_kpca_train = kpca.fit_transform(sim_train)
        X_kpca_test = kpca.transform(sim_test)

    elif model == "mean":
        # Mean imputation
        imputer = SimpleImputer(strategy='mean')

        # Fit the imputer on the training data and transform both training and test data
        X_kpca_train = imputer.fit_transform(train_X)
        X_kpca_test = imputer.transform(test_X)




    #  Calculate F1 score and accuracy
    # Initialize the RandomForest classifier
    rf = RandomForestClassifier(n_estimators=100, random_state=42)

    # Perform cross-validation
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    f1_scores = cross_val_score(rf, X_kpca_test, test_Y, cv=cv, scoring='f1_macro')
    acc_scores = cross_val_score(rf, X_kpca_test, test_Y, cv=cv, scoring='accuracy')

    # Print mean F1 score and accuracy
    print(f"Mean CV F1 Score {model}: {np.mean(f1_scores):.4f}")
    print(f"Mean CV Accuracy {model}: {np.mean(acc_scores):.4f}")

In [11]:

def run_test_reg(data,missing_rate,data_stats,mtype = "mcar"):
    X = data["X"]
    Y = data["Y"]
    train_X, test_X, train_Y, test_Y = train_test_split(X, Y, test_size=0.2, random_state=42)


    train_X = make_missing(train_X,rate = missing_rate,type = mtype)
    test_X = make_missing(test_X,rate = missing_rate,type = mtype)
    # modify_krn = Modify_Kernel(None, data_stats)
    # modify_krn.set_nbins(param_value)
    kernal = MKernel(None, data_stats)
    kernal.set_nbins(param_value)


    train_mod, test_mod = kernal.build_model(train_X, test_X)  # this does the pre-processing step

    sim_train = kernal.transform(train_mod)
    sim_test = kernal.transform(test_mod,train_mod)  # row = train, col = test


    # Configure Kernel PCA for precomputed kernels
    kpca = KernelPCA(kernel='precomputed')

    # Transform data using Kernel PCA
    X_kpca_train = kpca.fit_transform(sim_train)
    X_kpca_test = kpca.transform(sim_test)

    # Initialize the RandomForest regressor
    rf = RandomForestRegressor(n_estimators=100, random_state=42)

    # Perform cross-validation
    cv = KFold(n_splits=5, shuffle=True, random_state=42)
    mse_scores = cross_val_score(rf, X_kpca_test, test_Y, cv=cv, scoring='neg_mean_squared_error')
    mae_scores = cross_val_score(rf, X_kpca_test, test_Y, cv=cv, scoring='neg_mean_absolute_error')

    # Print mean MSE and MAE
    print(f"Mean Cross-Validation MSE: {-np.mean(mse_scores):.4f}")
    print(f"Mean Cross-Validation MAE: {-np.mean(mae_scores):.4f}")
